# Midterm 2025

1. **Mục tiêu:** Xây dựng mô hình phân lớp tiêu đề StackOverflow *(Phân lớp văn bản)* cho 3 loại: "hướng dẫn", "khái niệm" và "sửa lỗi".

2. **Yêu cầu:**
    - **Xử lý dữ liệu:**
        - Làm sạch dữ liệu (loại bỏ ký tự đặc biệt, chuyển về chữ thường, v.v.)
        - Chia dữ liệu thành tập huấn luyện, tập kiểm tra và tập xác thực
    - **Trích rút thông tin:**
        - Sử dụng các kỹ thuật như Bag of Words, TF-IDF hoặc Word Embeddings để chuyển đổi tiêu đề thành các đặc trưng số
    - **Huấn luyện và đánh giá mô hình:**
        - Chọn và huấn luyện các mô hình phân lớp (ví dụ: Logistic Regression, SVM, Random Forest, Neural Networks)
        - Đánh giá mô hình sử dụng các chỉ số như độ chính xác, độ nhạy, độ đặc hiệu và F1-score
        - So sánh hiệu suất của các mô hình khác nhau và chọn mô hình tốt nhất

3. **Dữ liệu:** Sử dụng tập dữ liệu tiêu đề StackOverflow đã được gán nhãn sẵn tại link: https://drive.google.com/uc?id=1ef8DpctKSDpQEZz5icfrKb9yk-b-jyWF

    Mô tả dữ liệu:
    - File: `question_title.csv`
    - Cột:
    - `title`: Tiêu đề câu hỏi
    - `label`: Nhãn phân loại (hướng dẫn, khái niệm, sửa lỗi)
    - Số lượng tiêu đề: 866 tiêu đề

4. **Hướng dẫn làm bài:**
    - Hoàn thành các câu hỏi trong notebook `midterm_2025.ipynb`
    - Hoàn thành code trong `### YOUR CODE HERE ###`
    - Không thay đổi các phần `# --- DO NOT CHANGE HERE ---` và `# --- IGNORE ---`

## 0. Cài đặt thư viện, dữ liệu

In [1]:
!pip install -q gdown scikit-learn

In [2]:
# Download dữ liệu
!gdown https://drive.google.com/uc?id=1ef8DpctKSDpQEZz5icfrKb9yk-b-jyWF

Downloading...
From: https://drive.google.com/uc?id=1ef8DpctKSDpQEZz5icfrKb9yk-b-jyWF
To: d:\Projects\TEST\nlp-course-lab\midterm\question_title.csv

  0%|          | 0.00/52.0k [00:00<?, ?B/s]
100%|██████████| 52.0k/52.0k [00:00<00:00, 3.02MB/s]


## 1. Đọc dữ liệu

### Câu 1: Đọc dữ liệu & Thống kê dữ liệu

**Mô tả:** Đọc dữ liệu từ file `question_title.csv` và hiển thị 5 dòng đầu tiên của dữ liệu

In [3]:
import pandas as pd

def read_data(file_path: str) -> pd.DataFrame:
    """
    Đọc dữ liệu từ file CSV
    Args:
        file_path (str): Đường dẫn tới file CSV
    Returns:
        pd.DataFrame: Dữ liệu được đọc từ file
    """
    ### YOUR CODE HERE
    data = pd.read_csv(file_path)

    return data
    ### YOUR CODE HERE

In [4]:
DATASET_PATH = 'question_title.csv' # Đường dẫn tới file dữ liệu

# --- DO NOT CHANGE HERE ---
data = read_data(file_path=DATASET_PATH)

# Print columns
print(f"Columns: {data.columns}")

# Hiển thị 5 dòng đầu tiên của dữ liệu
data.head()

# --- DO NOT CHANGE HERE ---

Columns: Index(['Unnamed: 0', 'question_title', 'question_type'], dtype='object')


,Unnamed: 0,question_title,question_type
0,0,What are metaclasses in Python?,0
1,1,Hidden features of Python,0
2,2,"What is the effect of extern ""C"" in C++?",0
3,3,When should I use Cross Apply over Inner Join?,0
4,4,Embedding Base64 Images,0


In [5]:
# Thống kê số lượng dữ liệu theo nhãn
def count_labels(data: pd.DataFrame) -> pd.Series:
    """
    Đếm số lượng dữ liệu theo nhãn
    Args:
        data (pd.DataFrame): Dữ liệu đầu vào, gồm các cột 'Unnamed: 0', 'question_title', 'question_type'
    Returns:
        pd.Series: Số lượng dữ liệu theo nhãn
    """
    ### YOUR CODE HERE
    label_counts = data['question_type'].value_counts()

    return label_counts

    ### YOUR CODE HERE

In [6]:
# --- DO NOT CHANGE HERE ---

label_counts = count_labels(data)
label_counts

# --- DO NOT CHANGE HERE ---

question_type
1    305
0    300
2    261
Name: count, dtype: int64

## 2. Xử lý dữ liệu

**Mô tả:** Tiền xử lý dữ liệu văn bản gồm các bước:
1. Loại bỏ các ký tự đặc biệt, số, dấu câu
2. Chuyển chữ in hoa thành in thường
3. Lemmatization
4. Xoá các từ có độ dài ngắn (ví dụ: <= 2 ký tự)

### Câu 2: Loại bỏ ký tự đặc biệt, số, dấu câu sử dụng biểu thức chính quy

In [7]:
import re

def regex(text: str) -> str:
    """
    Loại bỏ các ký tự đặc biệt, chữ số, khoảng trắng thừa sử dụng biểu thức chính quy
    Args:
        text (str): Văn bản đầu vào
    Returns:
        str: Văn bản đã được làm sạch
    """
    ### YOUR CODE HERE
    # Loại bỏ các ký tự đặc biệt và chữ số
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()

    return text
    ### YOUR CODE HERE

In [8]:
# --- DO NOT CHANGE HERE ---
regex(text="1.3_Million people    die in car accidents every year")

# --- DO NOT CHANGE HERE ---

'Million people die in car accidents every year'

### Câu 3: Chuyển chữ in hoa thành in thường

In [9]:
def text_lowercase(text: str) -> str:
    """
    Chuyển chữ in hoa thành in thường
    Args:
        text (str): Văn bản đầu vào
    Returns:
        str: Văn bản đã được chuyển thành chữ thường
    """
    ### YOUR CODE HERE
    text = text.lower()

    return text
    ### YOUR CODE HERE

In [10]:
# --- DO NOT CHANGE HERE ---
text_lowercase(text="Our AI Smart Mobility technology is making driving safer and more comfortable")

# --- DO NOT CHANGE HERE ---

'our ai smart mobility technology is making driving safer and more comfortable'

### Câu 4: Lemmatization

**Mô tả:** Sử dụng thư viện `nltk` để thực hiện chuyển đổi về dạng gốc của từ (lemmatization)

In [11]:
from nltk.stem import WordNetLemmatizer

# Define lemmatizer
lemitizer = WordNetLemmatizer()

def lemmatization(text: str) -> str:
    """
    Chuyển đổi về dạng gốc của từ (lemmatization) cho động từ
    Args:
        text (str): Văn bản đầu vào
    Returns:
        str: Văn bản đã được lemmatization
    """
    ### YOUR CODE HERE
    words = text.split()
    lemmatized_words = [lemitizer.lemmatize(word=word, pos='v') for word in words]
    lemmatized_text = ' '.join(lemmatized_words)

    return lemmatized_text
    ### YOUR CODE HERE

In [12]:
# --- DO NOT CHANGE HERE ---
sample_text = "saying said cars better bought does doors"
lemmatization(text=sample_text)

# --- DO NOT CHANGE HERE ---

'say say cars better buy do doors'

### Câu 5: Xoá các từ có độ dài bằng 1

**Mô tả:** Xoá các từ có độ dài ngắn (ví dụ: <= 2 ký tự) khỏi tiêu đề

In [13]:
def remove_short_words(text: str, min_length: int = 2) -> str:
    """
    Xoá các từ có độ dài ngắn (ví dụ: <= 2 ký tự) khỏi tiêu đề
    Args:
        text (str): Văn bản đầu vào
        min_length (int): Độ dài tối thiểu của từ để giữ lại
    Returns:
        str: Văn bản đã được loại bỏ các từ ngắn
    """
    ### YOUR CODE HERE
    words = text.split()
    filtered_words = [word for word in words if len(word) > min_length]
    filtered_text = ' '.join(filtered_words)

    return filtered_text
    ### YOUR CODE HERE

In [14]:
# --- DO NOT CHANGE HERE ---
remove_short_words(text="A car is a road vehicle, typically with four wheels, powered by an internal combustion engine or electric motor, and able to carry a small number of people", min_length=2)

# --- DO NOT CHANGE HERE ---

'car road vehicle, typically with four wheels, powered internal combustion engine electric motor, and able carry small number people'

### Câu 6: Hàm tiền xử lý dữ liệu

**Mô tả:** Viết hàm `preprocess_text` để thực hiện tất cả các bước tiền xử lý dữ liệu ở trên. Sau đó áp dụng hàm này cho cột `question_title` trong DataFrame và lưu kết quả vào cột mới `processed_question_title`

In [15]:
# Hàm tiền xử lý dữ liệu
def preprocess_text(text: str) -> str:
    """
    Tiền xử lý văn bản bao gồm các bước:
    1. Loại bỏ các ký tự đặc biệt, chữ số, khoảng trắng thừa
    2. Chuyển chữ in hoa thành in thường
    3. Lemmatization
    4. Xoá các từ có độ dài ngắn (ví dụ: <= 2 ký tự)
    """

    ### YOUR CODE HERE
    text = regex(text)
    text = text_lowercase(text)
    text = lemmatization(text)
    text = remove_short_words(text)

    return text
    ### YOUR CODE HERE

In [16]:
# --- DO NOT CHANGE HERE ---
preprocess_text(text="1.3_Million people    die in a car accidents every year")

# --- DO NOT CHANGE HERE ---

'million people die car accidents every year'

In [17]:
# Sử dụng hàm tiền xử lý dữ liệu để tiền xử lý cột "question_title"

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Tiền xử lý cột "question_title" trong DataFrame
    Args:
        df (pd.DataFrame): Dữ liệu đầu vào, gồm các cột 'Unnamed: 0', 'question_title', 'question_type'
    Returns:
        pd.DataFrame: Dữ liệu đã được tiền xử lý, thêm cột 'processed_question_title'
    """
    ### YOUR CODE HERE

    df["processed_question_title"] = df["question_title"].apply(func=preprocess_text)

    return df
    ### YOUR CODE HERE

In [18]:
# --- DO NOT CHANGE HERE ---
process_df = preprocess_data(df=data)

# Print 5 rows of processed data
process_df.head()

# --- DO NOT CHANGE HERE ---

,Unnamed: 0,question_title,question_type,processed_question_title
0,0,What are metaclasses in Python?,0,what metaclasses python
1,1,Hidden features of Python,0,hide feature python
2,2,"What is the effect of extern ""C"" in C++?",0,what the effect extern
3,3,When should I use Cross Apply over Inner Join?,0,when should use cross apply over inner join
4,4,Embedding Base64 Images,0,embed base image


## 3. Trích rút thông tin

**Mô tả:** Sử dụng TF-IDF để chuyển đổi tiêu đề thành các đặc trưng số

### Câu 7: Định nghĩa vectorizer

**Mô tả:** Sử dụng một trong các kỹ thuật trích xuất đặc trưng để trích xuất đặc trưng của `question_title` (Text embedding)


In [19]:
from typing import Tuple, Any

def define_vectorizer() -> Any:
    """
    Định nghĩa đối tượng vectorizer để trích xuất đặc trưng văn bản
    (Có thể sử dụng CountVectorizer, TfidfVectorizer, Word2Vec, FastText, GloVe,...)
    Returns:
        Any: Đối tượng vectorizer đã được định nghĩa
    """
    ### YOUR CODE HERE
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer(max_features=2000)

    return vectorizer
    ### YOUR CODE HERE

In [20]:
# --- DO NOT CHANGE HERE ---

test_vectorizer = define_vectorizer()
print(f"Vectorizer: {test_vectorizer}")

# Test sample text and fit transform
sample_texts = ["This is a sample text", "Another sample text for testing"]
sample_features = test_vectorizer.fit_transform(sample_texts)
print(f"Sample features shape: {sample_features.shape}")

# --- DO NOT CHANGE HERE ---

Vectorizer: TfidfVectorizer(max_features=2000)
Sample features shape: (2, 7)


## 4. Huấn luyện và đánh giá mô hình

**Mô tả:** 
- Huấn luyện và đánh giá mô hình phân lớp đối với 3 loại: "hướng dẫn", "khái niệm" và "sửa lỗi"
- Do số lượng data ít (<1000) nên sử dụng K-fold với n_fold = 10 để đánh giá mô hình

> Gợi ý: Có thể sử dụng các mô hình như Logistic Regression, SVM, Random Forest, Neural Networks


### Câu 8: Xây dựng mô hình để phân loại tiêu đề

**Mô tả:** 
- Xây dựng mô hình phân lớp để phân loại tiêu đề StackOverflow thành 3 loại: "hướng dẫn", "khái niệm" và "sửa lỗi". 
- Sử dụng K-fold cross-validation với n_fold = 10 để đánh giá mô hình. In ra các chỉ số đánh giá như accuracy, precision, recall và F1-score cho mỗi fold và trung bình các chỉ số này sau tất cả các fold.

In [21]:
from typing import Any

def define_classifier() -> Any:
    """
    Xây dựng và trả về một mô hình phân loại (classifier)
    Gợi ý: Mô hình có thể sử dụng Logistic Regression, SVM, Random Forest, Neural Networks,...

    Returns:
        Any: Mô hình phân loại (trả về một đối tượng mô hình đã được khởi tạo)
    """

    ### YOUR CODE HERE
    from sklearn.linear_model import LogisticRegression

    classifier = LogisticRegression(max_iter=1000, random_state=42, multi_class='ovr')

    return classifier
    ### YOUR CODE HERE

In [22]:
# --- DO NOT CHANGE HERE ---
classifier = define_classifier()

print(classifier)
# --- DO NOT CHANGE HERE ---

LogisticRegression(max_iter=1000, multi_class='ovr', random_state=42)


### Câu 9: Huấn luyện và đánh giá mô hình bằng K-fold cross-validation

**Mô tả:** Với mô hình đã xây dựng ở [Câu 8](#câu-8-xây-dựng-mô-hình-để-phân-loại-tiêu-đề), sử dụng K-fold cross-validation với n_fold = 10 để đánh giá mô hình. In ra các chỉ số đánh giá như accuracy, precision, recall và F1-score cho mỗi fold và trung bình các chỉ số này sau tất cả các fold.

In [23]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# --- DO NOT CHANGE HERE ---
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# --- DO NOT CHANGE HERE ---

def train_and_evaluate_model(vectorizer: Any, classifier: Any, X_text: pd.Series, y: pd.Series) -> None:
    """
    Huấn luyện và đánh giá mô hình sử dụng K-fold cross-validation
    Args:
        vectorizer (Any): Đối tượng vectorizer đã được định nghĩa
        classifier (Any): Mô hình phân loại (đối tượng mô hình đã được khởi tạo)
        X_text (pd.Series): Dữ liệu đầu vào (cột 'processed_question_title')
        y (pd.Series): Nhãn (cột 'question_type')
    """
    # --- DO NOT CHANGE HERE ---
    fold_accuracies = []
    fold_precisions = []
    fold_recalls = []
    fold_f1s = []
    # --- DO NOT CHANGE HERE ---


    ### YOUR CODE HERE
    for fold, (train_index, test_index) in enumerate(kf.split(X=X_text)):
        X_train_text, X_test_text = X_text.iloc[train_index], X_text.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        X_train = vectorizer.fit_transform(X_train_text)
        X_test = vectorizer.transform(X_test_text)

        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        fold_accuracies.append(accuracy)
        fold_precisions.append(precision)
        fold_recalls.append(recall)
        fold_f1s.append(f1)

        print(f"Fold {fold + 1}:")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-score: {f1:.4f}")

    ### YOUR CODE HERE


    # --- DO NOT CHANGE HERE ---
    avg_accuracy = np.mean(fold_accuracies)
    avg_precision = np.mean(fold_precisions)
    avg_recall = np.mean(fold_recalls)
    avg_f1 = np.mean(fold_f1s)

    print(f"Kết quả trung bình sau 10 folds:")
    print(f"  - Accuracy: {avg_accuracy:.4f}")
    print(f"  - Precision: {avg_precision:.4f}")
    print(f"  - Recall: {avg_recall:.4f}")
    print(f"  - F1-Score: {avg_f1:.4f}\n")
    # --- DO NOT CHANGE HERE ---

In [24]:
# --- DO NOT CHANGE HERE ---
vectorizer = define_vectorizer()
X = process_df['processed_question_title']
y = process_df['question_type']

train_and_evaluate_model(vectorizer=vectorizer, classifier=classifier, X_text=X, y=y)
# --- DO NOT CHANGE HERE ---

Fold 1:
  Accuracy: 0.6092
  Precision: 0.6203
  Recall: 0.6092
  F1-score: 0.6070
Fold 2:
  Accuracy: 0.6667
  Precision: 0.6686
  Recall: 0.6667
  F1-score: 0.6625
Fold 3:
  Accuracy: 0.6322
  Precision: 0.6348
  Recall: 0.6322
  F1-score: 0.6178
Fold 4:
  Accuracy: 0.6897
  Precision: 0.7006
  Recall: 0.6897
  F1-score: 0.6902
Fold 5:
  Accuracy: 0.6897
  Precision: 0.6917
  Recall: 0.6897
  F1-score: 0.6859
Fold 6:
  Accuracy: 0.7011
  Precision: 0.6984
  Recall: 0.7011
  F1-score: 0.6981
Fold 7:
  Accuracy: 0.7209
  Precision: 0.7338
  Recall: 0.7209
  F1-score: 0.7239
Fold 8:
  Accuracy: 0.5814
  Precision: 0.6172
  Recall: 0.5814
  F1-score: 0.5664
Fold 9:
  Accuracy: 0.5930
  Precision: 0.6052
  Recall: 0.5930
  F1-score: 0.5807
Fold 10:
  Accuracy: 0.5814
  Precision: 0.6152
  Recall: 0.5814
  F1-score: 0.5806
Kết quả trung bình sau 10 folds:
  - Accuracy: 0.6465
  - Precision: 0.6586
  - Recall: 0.6465
  - F1-Score: 0.6413



## 5. Lưu mô hình và vectorizer

In [25]:
import pickle
import os

# --- DO NOT CHANGE HERE ---
MODEL_DIR = "saved_model"
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

MODEL_PATH = os.path.join(MODEL_DIR, "text_classifier.sav")
VECTORIZER_PATH = os.path.join(MODEL_DIR, "tfidf_vectorizer.sav")

# Save model
pickle.dump(classifier, open(MODEL_PATH, 'wb'))
print(f"Model saved to '{MODEL_PATH}'")

# Save vectorizer
pickle.dump(vectorizer, open(VECTORIZER_PATH, 'wb'))
print(f"Vectorizer saved to '{VECTORIZER_PATH}'")

# --- DO NOT CHANGE HERE ---

Model saved to 'saved_model\text_classifier.sav'
Vectorizer saved to 'saved_model\tfidf_vectorizer.sav'


## 6. Sử dụng mô hình để dự đoán tiêu đề mới

### Câu 10: Dự đoán nhãn cho các tiêu đề mới

In [26]:
LABEL_MAP = {
    0: "khái niệm",
    1: "hướng dẫn",
    2: "sửa lỗi"
}

from typing import List

def infer_titles(titles: List[str]) -> List[str]:
    """
    Sử dụng mô hình đã lưu để dự đoán nhãn của tiêu đề mới
    Args:
        titles (List[str]): Danh sách tiêu đề mới cần dự đoán
    Returns:
        List[str]: Danh sách nhãn dự đoán tương ứng với các tiêu đề
    """
    ### YOUR CODE HERE
    processed_title = [preprocess_text(title) for title in titles]
    pred_labels = classifier.predict(vectorizer.transform(processed_title))
    pred_label_names = [LABEL_MAP[label] for label in pred_labels]

    return pred_label_names

    ### YOUR CODE HERE

In [27]:
# --- DO NOT CHANGE HERE ---
TEST_TITLE = [
    "scikit-learn random state in splitting dataset",
    "Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects",
    "how to learn from each fold in the k-fold cross validation?"
    "How to install tensorflow on windows 11",
    "what is the difference between list and tuple in python",
    "getting valueerror could not broadcast input array from shape"
]

pred_labels = infer_titles(titles=TEST_TITLE)
print("Predicted labels:", pred_labels)

# --- DO NOT CHANGE HERE ---

Predicted labels: ['hướng dẫn', 'sửa lỗi', 'hướng dẫn', 'khái niệm', 'sửa lỗi']
